In [1]:
import pyvisa

# Initialize the resource manager
rm = pyvisa.ResourceManager()

resources = []  # Initialize the resources variable

try:
    # List available resources
    resources = rm.list_resources()
    print(f'Available resources: {resources}')

    # Attempt to open each resource and identify the instrument
    for resource in resources:
        try:
            inst = rm.open_resource(resource)
            # Query the instrument ID to verify connection
            response = inst.query('*IDN?')
            print(f'Connected to: {response} at {resource}')

            # Query the current GPIB address
            current_address = inst.query('SYST:COMM:GPIB:ADDR?')
            print(f'Current GPIB Address: {current_address}')

            # Set a new GPIB address (example: setting to address 8)
            new_address = 8
            inst.write(f'SYST:COMM:GPIB:ADDR {new_address}')
            print(f'GPIB Address set to: {new_address}')

            # Close the resource
            inst.close()
            break  # Exit loop if connection is successful
        except pyvisa.errors.VisaIOError as e:
            print(f'Could not connect to {resource}: {e}')
            continue  # Try the next resource if connection fails
except pyvisa.errors.VisaIOError as e:
    print(f'Error listing resources: {e}')

if not resources:
    print('No VISA resources found.')



Available resources: ('ASRL1::INSTR', 'ASRL2::INSTR')
Could not connect to ASRL1::INSTR: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.
Could not connect to ASRL2::INSTR: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.


In [7]:
import pyvisa as visa
import pyvisa.constants as pyconst


class Communications:
    """
    This class offers the consumer a collection of wrapper menthods that
    leverage PyVisa calls and attempts to condense collections of methods
    therein while also adding in a means for echoing command calls to the
    terminal if the appropriate internal attribute is set to True. 

    Note that this is a work in progress and by no means a work of 
    perfection. Please feel free to copy, reuse, or enhance to your own
    liking and feel free to leave suggestions for improvement. Thanks!
    """

    def __init__(self, instrument_resource_string=None):
        self._instrument_resource_string = instrument_resource_string
        self._resource_manager = None
        self._instrument_object = None
        self._timeout = 20000
        self._echo_cmds = False
        self._version = 1.1

        try:
            if self._resource_manager is None:
                self._resource_manager = visa.ResourceManager()
        except visa.VisaIOError as visaerror:
            print(f"{visaerror}")
        except visa.VisaIOWarning as visawarning:
            print(f"{visawarning}")

    def connect(self, instrument_resource_string=None, timeout=None):
        """
        Open an instance of an instrument object for remote communication.

        Args:
            timeout (int): Time in milliseconds to wait before the \
                communication transaction with the target instrument\
                    is considered failed (timed out).
            
        Returns:
            None
        """
        try:
            if instrument_resource_string != None:
                self._instrument_resource_string = instrument_resource_string
                
            self._instrument_object = self._resource_manager.open_resource(
                self._instrument_resource_string
            )

            if timeout is None:
                self._instrument_object.timeout = self._timeout
            else:
                self._instrument_object.timeout = timeout
                self._timeout = timeout

            # Check for the SOCKET as part of the instrument ID string and set
            # the following accordingly...
            if "SOCKET" in self._instrument_resource_string:
                self._instrument_object.write_termination = "\n"
                self._instrument_object.read_termination = "\n"
                self._instrument_object.send_end = True

        except visa.VisaIOError as visaerr:
            print(f"{visaerr}")
        return

    def configure_rs232_settings(
        self,
        baudrate=19200,
        databits=8,
        parity=0,
        stopbits=1,
        flowcontrol=0,
        writetermination="\n",
        readtermination="\n",
        sendend=True,
    ):
        """
            This method pulls the collection of RS-232 settings together in 
            one location. For applicable PyVisa-specific constants that best
            align with this method's use, refer to documentation on
            pyvisa.constants. 

        Args:
            baudrate (int): Defines the baud rate to be used for data \
                transmission. Options are typically 2400, 4800, 9600, and \
                    others up to 115200. Refer to your instrument documentation \
                        for what is truly applicable. 
            databits (int): Typically 8, sometimes 7.
            parity (pyconst.Parity): Options include none, odd, and even.
            stopbits (pyconst.StopBits): Options include one and two.
            flowcontrol (pyconst.ControlFlow): Options include none, xon/xoff, \
                rts/cts, and dtr/dsr.
            read_terminator (str): Character options include "\\n" and "\\r".
            sendend (bool): Specifies whether or not the end character is to be \
                sent. 

        Returns:
            None
        """
        # First verify that the instrument resource string has "ASRL" so
        #   we know it connected as a serial instrument.
        if "ASRL" in self._instrument_resource_string:
            self._instrument_object.baud_rate = baudrate
            self._instrument_object.data_bits = databits
            if parity == 0:
                self._instrument_object.parity = pyconst.Parity.none
            elif parity == 1:
                self._instrument_object.parity = pyconst.Parity.odd
            elif parity == 2:
                self._instrument_object.parity = pyconst.Parity.even

            if stopbits == 0:
                self._instrument_object.stop_bits = pyconst.StopBits.one_and_a_half
            elif stopbits == 1:
                self._instrument_object.stop_bits = pyconst.StopBits.one
            elif stopbits == 2:
                self._instrument_object.stop_bits = pyconst.StopBits.two

            if flowcontrol == 0:
                self._instrument_object.flow_control = pyconst.ControlFlow.none
            elif flowcontrol == 1:
                self._instrument_object.flow_control = pyconst.ControlFlow.xon_xoff
            elif flowcontrol == 1:
                self._instrument_object.flow_control = pyconst.ControlFlow.rts_cts
            elif flowcontrol == 1:
                self._instrument_object.flow_control = pyconst.ControlFlow.dtr_dsr

            self._instrument_object.write_termination = writetermination
            self._instrument_object.read_termination = readtermination
            self._instrument_object.send_end = sendend

        else:
            print("raise an exception")

    def disconnect(self):
        """
        Close an instance of an instrument object.

        Args:
            None

        Returns:
            None
        """
        try:
            self._instrument_object.close()
        except visa.VisaIOError as visaerr:
            print(f"{visaerr}")
        return

    def write(self, command: str):
        """
        Issue controlling commands to the target instrument.

        Args:
            command (str): The command issued to the instrument to make it\
                perform some action or service.

        Returns:
            None
        """
        try:
            if self._echo_cmds is True:
                print(command)
            self._instrument_object.write(command)
        except visa.VisaIOError as visaerr:
            print(f"{visaerr}")
        return

    def read(self):
        """
        Used to read commands from the instrument.

        Args:
            None

        Returns:
            (str): The requested information returned from the target
            instrument.
        """
        return self._instrument_object.read()

    def query(self, command: str):
        """
        Used to send commands to the instrument  and obtain an information
        string from the instrument. Note that the information received will
        depend on the command sent and will be in string format.

        Args:
            command (str): The command issued to the instrument to make it
            perform some action or service.

        Returns:
            (str): The requested information returned from the target
            instrument.
        """
        response = ""
        try:
            if self._echo_cmds is True:
                print(command)
            response = self._instrument_object.query(command).rstrip()
        except visa.VisaIOError as visaerr:
            print(f"{visaerr}")

        return response

"""
Helps easily create GUIs for PyVISA instrumentation.

Provides class InstrumentOption which encapsulates information about a particular GUI option,
and function open_gui_return_input which takes in a list of InstrumentOptions, creates a GUI,
and returns the user's input when the GUI submit button is pressed.

GUI is formatted as a singular column of instrument options, with each option input presented 
as either a text box or check box.

Typical usage example:
    instrument_options = [
            InstrumentOption("High Current", "high_current", "0.001"),
            InstrumentOption("Filter Type", "filter_type"),
            InstrumentOption("Use Low to Earth", "lowToEarth_on", False, True),
            InstrumentOption("Use Input Capacitor", "inputCap_on", False, True),
        ]

    messages = "Message to display in GUI"

    parameters = open_gui_return_input(
        instrument_options, messages, "saved_parameters.txt"
    )


    Copyright 2023 Tektronix, Inc.                      
    See www.tek.com/sample-license for licensing terms. 

"""

import tkinter as tk
from tkinter import ttk

class InstrumentOption:
    """Class that encapsulates information about instrument parameters to present on GUI.

    A list of these can be passed to function open_gui_return_input() in order to automatically
    generate a Tkinter GUI window for getting instrument parameters from a user.

    Attributes:
        option_label: A string indicating the option name displayed in the GUI.
        gui_key: The key (string) used to retrieve the input value from Tkinter.
        default_value: A string (or bool if is_bool is True) indicating the default GUI value.
        is_bool: A boolean indicating if the GUI option should be a text field or boolean checkbox.
    """

    def __init__(
        self,
        label: str,
        gui_key: str,
        default_value="0",
        is_bool: bool = False,
        tooltip: str = "",
        permanent_tooltip=False,
    ) -> None:
        self.label = label
        self.gui_key = gui_key
        self.default_value = default_value
        self.is_bool = is_bool
        self.tooltip = tooltip
        self.permanent_tooltip = permanent_tooltip

    def tkinter_row(self, parent, label_length: int = 30):
        """Creates a Tkinter row for the instrument option input"""
        row = ttk.Frame(parent)
        
        # Create option label
        option_label = ttk.Label(row, text=self.label, width=label_length)
        option_label.pack(side=tk.LEFT)
        
        if self.is_bool:
            var = tk.BooleanVar(value=self.default_value)
            input_widget = ttk.Checkbutton(row, variable=var)
        else:
            var = tk.StringVar(value=self.default_value)
            input_widget = ttk.Entry(row, textvariable=var, width=10)
        
        input_widget.pack(side=tk.LEFT)
        
        # Create tooltip if permanent_tooltip is True
        if self.permanent_tooltip and self.tooltip:
            tooltip_label = ttk.Label(row, text=self.tooltip, font=("Arial", 10, "italic"))
            tooltip_label.pack(side=tk.LEFT)
        
        return row, var


def open_gui_return_input(instrument_options, messages: str, saved_parameters_filename: str):
    """Create GUI window with options specified by instrument_options, return input on user submit.

    Args:
        instrument_options (object): List of InstrumentOption objects to be added to GUI.

        messages (str): String that will be displayed as a note on the GUI window.

        saved_parameters_filename (str): Path name of file where user parameters will be saved
            to and reloaded from.

    Returns:
        (dict): Dictionary containing instrument parameters entered in the GUI, with keys determined
            by the gui_key attributes of the InstrumentOption objects in instrument_options.
    """
    # Open the parameters file or create one
    try:
        with open(saved_parameters_filename, "r", encoding="utf-8") as file:
            saved_params = file.read().splitlines()
    except FileNotFoundError:
        with open(saved_parameters_filename, "x", encoding="utf-8") as file:
            saved_params = []

    # Set GUI parameter default values to previously saved parameters
    if len(saved_params) == len(instrument_options):
        for i, value in enumerate(saved_params):
            if instrument_options[i].is_bool:
                instrument_options[i].default_value = True if value == "True" else False
            else:
                instrument_options[i].default_value = value

    # Create main Tkinter window
    root = tk.Tk()
    root.title("Instrument Parameter GUI")

    # Get max length of any instrument_option label, used for alignment
    max_label_length = max(len(option.label) for option in instrument_options)

    # Create main frame
    main_frame = ttk.Frame(root, padding="10")
    main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

    # Create and add instrument option rows
    var_dict = {}
    for i, option in enumerate(instrument_options):
        row, var = option.tkinter_row(main_frame, label_length=max_label_length)
        row.grid(row=i, column=0, sticky=tk.W, pady=2)
        var_dict[option.gui_key] = var

    # Create message label
    message_label = ttk.Label(main_frame, text=messages, wraplength=300, justify=tk.CENTER)
    message_label.grid(row=len(instrument_options), column=0, pady=10)

    # Create submit button
    def on_submit():
        with open(saved_parameters_filename, "w", encoding="utf-8") as file:
            for key, var in var_dict.items():
                file.write(str(var.get()) + "\n")
        root.quit()

    submit_button = ttk.Button(main_frame, text="Run Test", command=on_submit)
    submit_button.grid(row=len(instrument_options) + 1, column=0, pady=10)

    root.mainloop()

    return {key: var.get() for key, var in var_dict.items()}

"""Keithley PyVisa example code that connects to 6221 + 2182/2182A instrument
stack and runs delta measurements with user parameters from GUI.

Set INSTRUMENT_RESOURCE_STRING equal to your instrument's resource string, found using the
    VISA Interactive Control program.

Note: 6221 Buffer can only store a max of 65,536 readings, limiting the amount of data this
program can collect.

    Copyright 2023 Tektronix, Inc.                      
    See www.tek.com/sample-license for licensing terms.
"""
import os
import time
import datetime
import textwrap


DEBUG_PRINT_COMMANDS = True

# Ensure the parameters file is in the current working directory
SAVED_PARAMETERS_FILENAME = os.path.join(os.getcwd(), "sweep_parameters.txt")

def experiment_setup(instrument, parameters, start_time):
    """Set up instrument parameters for linear sweep test"""
    filter_type = int(parameters["filter_type"])
    filter_count = int(parameters["filter_count"])
    integration_nplcs = float(parameters["integration_NPLCs"])
    volt_compliance = float(parameters["volt_compliance"])
    guarding_on = parameters["guarding_on"]
    low_to_earth_on = parameters["lowToEarth_on"]
    #sweepdelay = float(parameters["sweepdelay"])
    pulsehighcurrent = float(parameters["pulsehighcurrent"])
    pulselowcurrent = float(parameters["pulselowcurrent"])
    pulsecount = int(parameters["pulsecount"])
    pulsesweeprange = parameters["pulsesweeprange"]
    pulsewidth = float(parameters["pulsewidth"])
    pulsedelay = float(parameters["pulsedelay"])
    pulsesweepstate = int(parameters["pulsesweepstate"])
    pulselowmeas = int(parameters["pulselowmeas"])
    pulseinterval = float(parameters["pulseinterval"])
    pulsecompliance = float(parameters["pulsecompliance"])
    pulsesweepspacing = parameters["pulsesweepspacing"]
    pulsesweepstart = float(parameters["pulsesweepstart"])
    pulsesweepstop = float(parameters["pulsesweepstop"])
    pulsesweepstep = float(parameters["pulsesweepstep"])
    pulsevoltagerange = float(parameters["pulsevoltagerange"])
    pulsefilterstate = int(parameters["pulsefilterstate"])
    #pulsefiltercount = int(parameters["pulsefiltercount"])
    #pulsefiltertype = int(parameters["pulsefiltertype"])
    #num_readings = pulsecount + (pulsecount * pulselowmeas)
    
    query2182Apresent = int(instrument.query("sour:pdel:nvpresent?"))
    if query2182Apresent == 0:
        print("2182A not present")
        instrument.disconnect()
        stop_time = time.time()  # Stop the timer...
        print(f"Elapsed Time: {(stop_time - start_time):0.3f}s")
        exit()

    invalid_parameters = False
    if pulsehighcurrent == pulselowcurrent:
        print("High and low current cannot be the same")
        invalid_parameters = True
    if pulsecount < 1 or pulsecount > 65536:
        print("Number of readings must be in range [1, 65536]")
        invalid_parameters = True
    if filter_type not in [0, 1, 2]:
        print("Filter type must be 0, 1, or 2")
        invalid_parameters = True
    if (filter_count < 2 or filter_count > 300) and filter_type != 0:
        print("If filter is enabled, filter count must be in range [2, 300]")
        invalid_parameters = True
    if pulsedelay < 1e-3 or pulsedelay > 9999.999:
        print("Delay must be in range [1e-3, 9999.999]")
        invalid_parameters = True
    if volt_compliance < 0.1 or volt_compliance > 105:
        print("Voltage compliance must be in range [0.1, 105]")
        invalid_parameters = True

    if invalid_parameters:
        instrument.disconnect()
        stop_time = time.time()  # Stop the timer...
        print(f"Elapsed Time: {(stop_time - start_time):0.3f}s")
        exit()

    # Abort tests in the other modes
    instrument.write("SOUR:SWE:ABOR")
    instrument.write("SOUR:WAVE:ABOR")

    instrument.write("*RST")  # Reset the 6221
    time.sleep(2) # Wait 2 seconds
    instrument.write("SYST:COMM:SERIAL:SEND \"*RST\"")  # Reset the 2182A
    time.sleep(3) # Wait 3 seconds
    # Use Guard?
    if guarding_on:
        instrument.write("OUTP:ISH GUARD")
    else:
        instrument.write("OUTP:ISH OLOW")

    # Low to Earth?
    if low_to_earth_on:
        instrument.write("OUTP:LTE ON")
    else:
        instrument.write("OUTP:LTE OFF")

    time.sleep(0.5)

    sweepdelay = ((pulsecount * 1/50 - 6)/1000)
    
    instrument.write(f"SOUR:DEL {sweepdelay}") # Set delay between pulses
    instrument.write("form:elem READ,TST,RNUM,SOUR")  # Set readings to be returned
    instrument.write(f"SOUR:CURR:COMP {volt_compliance}")  # Set voltage compliance
    instrument.write(f"sour:pdel:high {pulsehighcurrent}")  # Set high current
    instrument.write(f"sour:pdel:low {pulselowcurrent}")  # Set low current
    instrument.write(f"sour:pdel:count {pulsecount}")  # Set num readings
    instrument.write(f"sour:pdel:rang best")  # Best uses lowest range which will handle all sweep steps
    instrument.write(f"sour:swe:rang {pulsesweeprange}")  # Set sweep range
    instrument.write(f"sour:pdel:width {pulsewidth}")  # Set pulse width
    instrument.write(f"sour:pdel:sdel {pulsedelay}")  # Set pulse delay
    instrument.write(f"sour:pdel:swe {pulsesweepstate}")  # Set pulse sweep state
    instrument.write(f"sour:pdel:lme {pulselowmeas}")  # Set # of low measurements
    instrument.write(f"sour:pdel:int {pulseinterval}")  # Set pulse interval
    instrument.write(f"sour:curr:comp {pulsecompliance}")  # Set pulse compliance
    instrument.write(f"sour:swe:spac {pulsesweepspacing}")  # Set pulse sweep spacing
    instrument.write(f"sour:curr:start {pulsesweepstart}")  # Set pulse sweep start
    instrument.write(f"sour:curr:stop {pulsesweepstop}")  # Set pulse sweep stop
    instrument.write(f"sour:curr:step {pulsesweepstep}")  # Set pulse sweep step
    instrument.write(f"SYST:COMM:SERIAL:SEND \":sens:volt:rang {pulsevoltagerange}\"")  # Set voltage measure range
    instrument.write(f"sens:aver:wind 0")  # Set averaging window
    instrument.write(f"sens:aver:stat {pulsefilterstate}")  # Set filter state
    #instrument.write(f"sens:aver:count {pulsefiltercount}")  # Set filter count
    #instrument.write(f"sens:aver:tcon {pulsefiltertype}")  # Set filter type
    instrument.write(f"SYST:COMM:SERIAL:SEND \":SENS:VOLT:NPLC {integration_nplcs}\"")  # Set NPLC for 2182A
    time.sleep(0.5)

    # Set the filter type
    if filter_type == 0:
        instrument.write("SENS:AVER:TCON MOV")
        instrument.write("SENS:AVER:STAT 0")  # turn off filter
    elif filter_type == 1:
        instrument.write("SENS:AVER:TCON MOV")
        instrument.write(f"SENS:AVER:COUNT {filter_count}")  # set filter count
        instrument.write("SENS:AVER:STAT 1")  # turn on filter
    elif filter_type == 2:
        instrument.write("SENS:AVER:TCON REP")
        instrument.write(f"SENS:AVER:COUNT {filter_count}")  # set filter count
        instrument.write("SENS:AVER:STAT 1")  # turn on filter
    else:
        print("Error: Invalid filter type parameter")
        exit()
    time.sleep(2)
    instrument.write("UNIT V")  # Set units (other options are OHMS, W, SIEM)

def read_data(instrument, num_readings: int):
    """Wait for instruments to finish measuring and read data"""
    # Wait for sweep to finish
    sweep_done = False
    data = []
    while not sweep_done:
        time.sleep(1)
        oper_byte_status = instrument.query(
            "STAT:OPER:EVEN?"
        )  # Check the event register
        oper_byte_status = int(oper_byte_status)  # Convert decimal string to int
        sweep_done = oper_byte_status & (1 << 1)  # bit mask to check if sweep done

    # Get the measurements 1000 at a time
    for i in range((num_readings - 1) // 1000 + 1):
        if num_readings - i * 1000 > 1000:  # if more than 1000 readings left
            data_sel_count = 1000
        else:
            data_sel_count = num_readings - i * 1000

        # Transfer readings over GPIB
        raw_data = instrument.query(f"TRAC:DATA:SEL? {i * 1000},{data_sel_count}")
        raw_data = raw_data.split(",")
        data.extend(raw_data)

    return data

def write_csv(data, csv_path: str):
    """Write data to csv file"""
    # Create and open file
    with open(csv_path, "a+", encoding="utf-8") as csv_file:
        # Write the measurements to the CSV file
        csv_file.write(
            "Voltage Reading, Timestamp, Source Current, Reading Number, Resistance\n"
        )

        # We queried 4 values per reading, so iterate over groups of 4 elements
        for i in range(0, len(data), 4):
            v_reading = data[i]
            time_stamp = data[i + 1]
            source_current = data[i + 2]
            reading_number = data[i + 3]
            resistance = float(v_reading) / float(source_current)
            csv_file.write(
                f"{v_reading}, {time_stamp}, {source_current}, {reading_number}, {resistance}\n"
            )
            csv_file.flush()

def main():
    """Main function. Connect to instrument, get test params through GUI, run test, save data."""
    start_time = time.time()  # Start the timer...
    inst_6221 = Communications("GPIB0::12::INSTR")
    inst_6221.connect()

    # Options to be displayed in GUI
    instrument_options = [
        InstrumentOption(
            "Pulse High Current",
            "pulsehighcurrent",
            "0.001",
            tooltip="Sets high pulse value (amps). [-105e-3 to 105e-3]",
        ),
        InstrumentOption(
            "Pulse Low Current",
            "pulselowcurrent",
            "0",
            tooltip="Sets low pulse value (amps). [-105e-3 to 105e-3]",
        ),
        InstrumentOption(
            "Pulse Sweep Spacing",
            "pulsesweepspacing",
            "LIN",
            tooltip="LIN = Linear, LOG = Logarithmic",
        ),
        InstrumentOption(
            "Pulse Sweep Start",
            "pulsesweepstart",
            "0.001",
            tooltip="Initial value of the pulse sweep",
        ),
        InstrumentOption(
            "Pulse Sweep Stop",
            "pulsesweepstop",
            "0.01",
            tooltip="Final value of the pulse sweep",
        ),
        InstrumentOption(
            "Pulse Sweep Step",
            "pulsesweepstep",
            "0.001",
        ),
        InstrumentOption(
            "Number of Pulses",
            "pulsecount",
            "11",
            tooltip="Number of pulses in the sweep including the start and stop values",
        ),
        InstrumentOption(
            "Pulse Sweep Range",
            "pulsesweeprange",
            "best",
            tooltip="Best or Auto range for the pulse sweep",
        ),
        InstrumentOption(
            "Pulse Width",
            "pulsewidth",
            "0.001",
            tooltip="Width of the pulse",
        ),
        InstrumentOption(
            "Pulse Sweep State",
            "pulsesweepstate",
            "0",
            tooltip="0 = Off, 1 = On",
        ),
        InstrumentOption(
            "Pulse Low Measurement",
            "pulselowmeas",
            "0",
            tooltip="Set number of low measurements. NRf = 1 or 2",
        ),
        InstrumentOption(
            "Pulse Interval (sec)",
            "pulseinterval",
            "0.5",
            tooltip="Interval between pulses (pulse duty cycle = pulse / (interval between pulses + pulse width))",
        ),
        InstrumentOption(
            "Pulse Compliance",
            "pulsecompliance",
            "10",
            tooltip="Voltage compliance must be in range [0.1, 105]",
        ),
        InstrumentOption(
            "Pulse Delay",
            "pulsedelay",
            "0.5",
            tooltip="Pulse settle time delay between current source change and measurement trigger.",
        ),
        InstrumentOption(
            "Filter Type",
            "filter_type",
            tooltip="0 = No Filter, 1 = Moving Filter, 2 = Repeat Filter",
        ),
        InstrumentOption(
            "Filter Count",
            "filter_count",
            "2",
            tooltip="Must be in range [2, 300] if filter in use",
        ),
        InstrumentOption(
            "Pulse Filter State",
            "pulsefilterstate",
            "0",
            tooltip="0 = Off, 1 = On",
        ),
        InstrumentOption(
            "Pulse Voltage Range",
            "pulsevoltagerange",
            "0.1",
            tooltip="Valid Voltage Ranges: 0.01, 0.1, 1, 10 or 100",
        ),
        InstrumentOption("Integration NPLCs", "integration_NPLCs", "5"),
        InstrumentOption(
            "Voltage Compliance",
            "volt_compliance",
            "10",
            tooltip="Voltage compliance must be in range [0.1, 105]",
        ),
        InstrumentOption("Use Guard", "guarding_on", False, True),
        InstrumentOption("Use Low to Earth", "lowToEarth_on", False, True),
    ]

    messages = (
        "Test Setup: Connect the 6221 Current Source and 2182 Nanovoltmeter "
        "through their RS-232 ports and connect the 6221 to this PC via GPIB adapter."
    )

    # Wrap messages so each line is no more than 40 characters.
    messages = "\n".join(textwrap.wrap(messages, 40))

    parameters = open_gui_return_input(
        instrument_options, messages, SAVED_PARAMETERS_FILENAME
    )

    # If user clicks close button on app window without clicking run
    if list(parameters.values())[0] is None:
        print("Application window closed without starting test; aborting")
        inst_6221.disconnect()
        stop_time = time.time()  # Stop the timer...

        print(f"Elapsed Time: {(stop_time - start_time):0.3f}s")
        exit()

    pulsecount = int(parameters["pulsecount"])
    pulselowmeas = int(parameters["pulselowmeas"])
    num_readings = pulsecount + (pulsecount * pulselowmeas)
    
    experiment_setup(inst_6221, parameters, start_time)

    inst_6221.write("sour:pdel:arm")  # Arm the test
    time.sleep(3)
    inst_6221.write("init:imm")  # Start the test

    data = read_data(inst_6221, num_readings)

    date = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    csv_path = f".\\PulsedIVLinear_Measurements {date}.csv"
    write_csv(data, csv_path)

    inst_6221.write(
        ":SOUR:SWE:ABORT"
    )  # Abort the test to prevent the output being left on

    inst_6221.disconnect()

    stop_time = time.time()  # Stop the timer...

    # Notify the user of completion and the data streaming rate achieved.
    print("done")
    print(f"Elapsed Time: {(stop_time - start_time):0.3f}s")

    exit()

if __name__ == "__main__":
    main()

"""
    Copyright 2023 Tektronix, Inc.                      
    See www.tek.com/sample-license for licensing terms. 
"""



VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.


AttributeError: 'NoneType' object has no attribute 'query'

: 

In [ ]:
import pyvisa

# Initialize the resource manager with the pyvisa-py backend
rm = pyvisa.ResourceManager('@py')

try:
    # List available resources
    resources = rm.list_resources()
    print(f'Available resources: {resources}')

    # Attempt to open each resource and identify the instrument
    for resource in resources:
        try:
            inst = rm.open_resource(resource)
            # Query the instrument ID to verify connection
            response = inst.query('*IDN?')
            print(f'Connected to: {response} at {resource}')

            # Query the current GPIB address
            current_address = inst.query('SYST:COMM:GPIB:ADDR?')
            print(f'Current GPIB Address: {current_address}')

            # Set a new GPIB address (example: setting to address 8)
            new_address = 8
            inst.write(f'SYST:COMM:GPIB:ADDR {new_address}')
            print(f'GPIB Address set to: {new_address}')

            # Close the resource
            inst.close()
            break  # Exit loop if connection is successful
        except pyvisa.errors.VisaIOError as e:
            print(f'Could not connect to {resource}: {e}')
            continue  # Try the next resource if connection fails
except pyvisa.errors.VisaIOError as e:
    print(f'Error listing resources: {e}')

if not resources:
    print('No VISA resources found.')


Available resources: ()
No VISA resources found.


In [ ]:
import pyvisa

# Initialize the resource manager using the NI backend
rm = pyvisa.ResourceManager()

try:
    # List available resources
    resources = rm.list_resources()
    print(f'Available resources: {resources}')

    if resources:
        # Open the first available resource
        inst = rm.open_resource(resources[0])

        # Query the instrument ID to verify connection
        response = inst.query('*IDN?')
        print(f'Connected to: {response} at {resources[0]}')

        # Close the resource
        inst.close()
    else:
        print('No VISA resources found.')
except pyvisa.errors.VisaIOError as e:
    print(f'Error: {e}')


Error: VI_ERROR_INV_EXPR (-1073807344): Invalid expression specified for search.


In [ ]:
import pyvisa

# Initialize the resource manager
rm = pyvisa.ResourceManager()

try:
    # Use a more general query to list all instruments
    resources = rm.list_resources('TCP*')
    print(f'Available resources: {resources}')
    
    for resource in resources:
        print(f'Trying resource: {resource}')
        try:
            inst = rm.open_resource(resource)
            response = inst.query('*IDN?')
            print(f'Connected to: {response} at {resource}')
            inst.close()
        except Exception as e:
            print(f'Error connecting to {resource}: {e}')
except Exception as e:
    print(f'Error listing resources: {e}')


Error listing resources: VI_ERROR_INV_EXPR (-1073807344): Invalid expression specified for search.
